In [17]:
import numpy as np
from datetime import datetime
import requests
import pandas as pd
import matplotlib.pyplot as plt

In [18]:
url = "http://vps-40d69db1.vps.ovh.ca:21046/api/ExecucaoSimulacaos/melhorValidacaoPeriodoFlat?idPeriodo=9"

In [19]:
r = requests.get(url)
j = r.json()
df = pd.DataFrame.from_dict(j)

In [20]:
df.columns

Index(['simulacaoId', 'ticker', 'simulacaoResultado', 'simulacaoLucro',
       'simulacaoPrejuizo', 'target', 'stop', 'tipo', 'experimentoSimulacaoId',
       'validacaoLucro', 'validacaoPrejuizo', 'saldoValidacao'],
      dtype='object')

Qual o período ?
Quantos experimentos ?
Total de casos ?

In [24]:
dfResumo = df.groupby('experimentoSimulacaoId').count()
dfResumo = dfResumo.filter(items=['ticker', 'validacaoLucro','saldoValidacao'])
dfResumo['percentual'] = (dfResumo['validacaoLucro'] / dfResumo['ticker']) * 100
dfResumo.head()

,ticker,validacaoLucro,saldoValidacao,percentual
experimentoSimulacaoId,,,,
54,14458,14422,14422,99.751003
56,11339,11329,11329,99.911809
57,27537,27528,27528,99.967317
58,13297,0,0,0.000000


Tickers 

In [30]:
print("Quantidade total de tickers: %2d "%(len(df['ticker'].unique())))

Quantidade total de tickers: 60 


In [31]:
dfTicker = df.groupby(['ticker','saldoValidacao']).size().reset_index(name ='qtde')

In [32]:
dfTicker.head()

,ticker,saldoValidacao,qtde
0,AERI3,0.0,227
1,AERI3,1.0,22
2,AERI3,2.0,50
3,ALPA4,0.0,695
4,ALSO3,0.0,21


In [33]:
print("Quantidade de tickers já tratados: %2d "%(len(dfTicker['ticker'].unique())))

Quantidade de tickers já tratados: 46 


In [57]:
dfTicker2 = dfTicker.pivot_table('qtde','ticker','saldoValidacao')
dfTicker2.fillna(0, inplace=True)
dfTicker2['positivo'] = dfTicker2[1.0] + dfTicker2[2.0]
dfTicker2['negativo'] = dfTicker2[-1.0] + dfTicker2[-2.0] + dfTicker2[-3.0]
dfTicker2.drop(columns=[-3.0, -2.0, -1.0, 1.0, 2.0], inplace=True)
dfTicker2.head(11)

saldoValidacao,0.0,positivo,negativo
ticker,,,
AERI3,227.0,72.0,0.0
ALPA4,695.0,0.0,0.0
ALSO3,21.0,120.0,0.0
AMAR3,658.0,155.0,261.0
ANIM3,364.0,68.0,1182.0
B3SA3,3300.0,405.0,0.0
BKBR3,705.0,1107.0,0.0
BPAC11,5371.0,0.0,2217.0
BRDT3,4330.0,0.0,0.0


In [53]:

dfBom = dfTicker2[(dfTicker2['negativo']==0) & (dfTicker2['positivo']>0)]

In [54]:
len(dfBom)

12

In [55]:
dfBom.head(25)

saldoValidacao,0.0,positivo,negativo
ticker,,,
AERI3,227.0,72.0,0.0
ALSO3,21.0,120.0,0.0
B3SA3,3300.0,405.0,0.0
BKBR3,705.0,1107.0,0.0
BRML3,2280.0,72.0,0.0
ELET3,126.0,435.0,0.0
ENEV3,302.0,147.0,0.0
HAPV3,44.0,53.0,0.0
LIGT3,10.0,306.0,0.0


# Relacionar isso com os dados de regra-projecao.

### <<dataframe>>resultadoTickerComRegraProjecao(in regraProjecao, in dataInicio)